In [4]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    회사명 = inputs["회사명"]
    return ddg.run(f"{회사명}의 주식 심볼")


def get_income_statement(inputs):
    심볼 = inputs["심볼"]
    주식 = yfinance.Ticker(심볼)
    return json.dumps(주식.income_stmt.to_json())


def get_balance_sheet(inputs):
    심볼 = inputs["심볼"]
    주식 = yfinance.Ticker(심볼)
    return json.dumps(주식.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    심볼 = inputs["심볼"]
    주식 = yfinance.Ticker(심볼)
    return json.dumps(주식.history(period="3mo").to_json())


functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,
}

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "회사 이름을 입력받아 주식 심볼을 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "회사명": {
                        "type": "string",
                        "description": "검색할 회사의 이름",
                    }
                },
                "required": ["회사명"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "주식 심볼을 입력받아 해당 회사의 손익계산서를 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "심볼": {
                        "type": "string",
                        "description": "회사 주식 심볼",
                    },
                },
                "required": ["심볼"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "주식 심볼을 입력받아 해당 회사의 대차대조표를 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "심볼": {
                        "type": "string",
                        "description": "회사 주식 심볼",
                    },
                },
                "required": ["심볼"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "주식 심볼을 입력받아 최근 3개월 동안의 주식 성과를 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "심볼": {
                        "type": "string",
                        "description": "회사 주식 심볼",
                    },
                },
                "required": ["심볼"],
            },
        },
    },
]

In [5]:
# from dotenv import load_dotenv
# import os

# load_dotenv()
# api_key = os.getenv('OPENAI_API_KEY')

# from openai import OpenAI
# client = OpenAI()

# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     model="gpt-4-1106-preview",
#     tools=functions,
# )
# assistant

In [11]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

from openai import OpenAI
client = OpenAI()

assistant = client.beta.assistants.create(
    name="주식 분석 도우미", 
    instructions="당신은 사용자에게 상장된 회사들에 대해 조사하고 그 주식을 사야 할지 결정하는 데 도움을 줍니다.", 
    model="gpt-4-turbo",
    tools=functions,
)
assistant_id = assistant.id

In [24]:

thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "삼성전자의 주식 정보를 알고 싶어요.",
        }
    ]
)


In [25]:
# run 생성: assistant_id와 thread_id를 사용하여 대화 실행. 이로 인해 function_call이 실행될 수 있음.
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_BqQ7Jzo4ap1iPZye3YXvuNSQ', assistant_id='asst_Gz8k8Swf4qKwmQ79U4Dg9vQj', cancelled_at=None, completed_at=None, created_at=1726372323, expires_at=1726372923, failed_at=None, incomplete_details=None, instructions='당신은 사용자에게 상장된 회사들에 대해 조사하고 그 주식을 사야 할지 결정하는 데 도움을 줍니다.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-turbo', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_U4pf2roJkEmfLYBdqm7VFMts', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='회사 이름을 입력받아 주식 심볼을 반환합니다.', parameters={'type': 'object', 'properties': {'회사명': {'type': 'string', 'description': '검색할 회사의 이름'}}, 'required': ['회사명']}, strict=False), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description='주식 심볼을 입력받아 해당 회사의 손익계산서를 반환합니다.', parameters={'type': 'object', 'properties

In [26]:
# get_run 함수: 주어진 run_id와 thread_id에 대한 실행 정보를 가져오는 함수.
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )

# send_message 함수: 사용자로부터 새로운 메시지를 주어진 thread_id로 전송하는 함수.
def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )

# get_messages 함수: 주어진 thread_id의 모든 메시지를 가져와 출력.
def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")

# get_tool_outputs 함수: run_id와 thread_id에 있는 실행(run)에서 호출된 도구의 출력을 가져와 출력.
def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread.id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"호출 중인 함수: {function.name} 인자: {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs

# submit_tool_outputs 함수: 가져온 도구의 출력 결과를 OpenAI API에 제출하여 실행 결과를 업데이트.
def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id, thread_id=thread_id, tool_outputs=outpus
    )

In [27]:
submit_tool_outputs(run.id, thread.id)


호출 중인 함수: get_ticker 인자: {"회사명": "삼성전자"}
호출 중인 함수: get_income_statement 인자: {"심볼": "005930.KS"}
호출 중인 함수: get_balance_sheet 인자: {"심볼": "005930.KS"}
호출 중인 함수: get_daily_stock_performance 인자: {"심볼": "005930.KS"}


Run(id='run_BqQ7Jzo4ap1iPZye3YXvuNSQ', assistant_id='asst_Gz8k8Swf4qKwmQ79U4Dg9vQj', cancelled_at=None, completed_at=None, created_at=1726372323, expires_at=1726372923, failed_at=None, incomplete_details=None, instructions='당신은 사용자에게 상장된 회사들에 대해 조사하고 그 주식을 사야 할지 결정하는 데 도움을 줍니다.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-turbo', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1726372324, status='queued', thread_id='thread_U4pf2roJkEmfLYBdqm7VFMts', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='회사 이름을 입력받아 주식 심볼을 반환합니다.', parameters={'type': 'object', 'properties': {'회사명': {'type': 'string', 'description': '검색할 회사의 이름'}}, 'required': ['회사명']}, strict=False), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description='주식 심볼을 입력받아 해당 회사의 손익계산서를 반환합니다.', parameters={'type': 'object', 'prop

In [48]:
get_run(run.id, thread.id).status


'completed'

In [49]:
get_messages(thread.id)


user: 삼성전자의 주식 정보를 알고 싶어요.
assistant: ### 삼성전자 주식 정보 (심볼: 005930.KS)

#### 1. 주식 성과 (최근 3개월)
- 최근 주가 변동성이 높으며 트레이딩 범위가 넓음. 최고가는 88,800원, 최저가는 64,200원이었습니다.
- 최근에는 71,400원에서 마감하며 하락세를 보였습니다.

#### 2. 손익계산서 요약
- 매출액: 302,231.36 억원
- EBITDA: 86,303.14 억원
- 순이익: 54,730.02 억원

#### 3. 대차대조표 요약
- 총자산: 448,424.51 억원
- 총부채: 93,674.90 억원
- 총자본: 354,749.60 억원

#### 4. 투자 결정요인
삼성전자는 지속적으로 강한 실적을 보여 주고 있으며, 글로벌 반도체 시장에서의 위상이 매우 높습니다. 다만, 최근 금융시장 변동성과 글로벌 경제 불확실성으로 인해 단기적 주가 변동이 클 수 있습니다.

#### 투자 전략
- 장기 투자:삼성전자의 안정적인 사업 운영과 강력한 시장 지위를 고려해 장기적으로 투자 가치가 있음. 
- 단기 투자:시장 변동성을 활용한 단기 트레이딩이 가능하지만 높은 변동성을 감수해야 함.

#### 결론
삼성전자는 장기 투자자에게는 매력적인 투자 대상이 될 수 있으나, 단기 투자자는 시장 상황을 면밀히 모니터링할 필요가 있습니다. 또한 글로벌 경제 상황과 기술 분야에서 발생할 변화에 유의하면서 투자를 고려하는 것이 좋습니다.
